# Introduction to Multi GPU Programming with MPI and OpenACC

In this self-paced, hands-on lab, you will learn how to program multi GPU systems or GPU clusters using the Message Passing Interface (MPI) and OpenACC. Basic knowledge of MPI and OpenACC is a prerequisite. The topics covered by this lab are:

* Exchanging data between different GPUs using CUDA-aware MPI and OpenACC
* Handle GPU affinity in multi GPU systems
* Overlapping communication with computation to hide communication times
* Optionally how to use the NVIDIA performance analysis tools

Lab created by Jiri Kraus (based on the Lab **Accelerating C/C++ code with Multi-GPUs using CUDA** from Justin Luitjens and Mark Ebersole)

The following timer counts down to a five minute warning before the lab instance shuts down.  You should get a pop up at the five minute warning reminding you to save your work!

<iframe id="timer" src="timer/timer.html" width="100%" height="120px"></iframe>

---
Before we begin, let's verify [WebSockets](http://en.wikipedia.org/wiki/WebSocket) are working on your system.  To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above.  If all goes well, you should see some output returned below the grey cell.  If not, please consult the [Self-paced Lab Troubleshooting FAQ](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) to debug the issue.

In [ ]:
print "The answer should be three: " + str(1+2)

Let's execute the cell below to display information about the GPUs running on the server.

In [ ]:
!nvidia-smi

---
The following video will explain the infrastructure we are using for this self-paced lab, as well as give some tips on it's usage.  If you've never taken a lab on this system before, it's highly recommended that you watch this short video first.<br><br>
<div align="center"><iframe width="640" height="390" src="http://www.youtube.com/embed/ZMrDaLSFqpY" frameborder="0" allowfullscreen></iframe></div>

# Motivation

## Why use Multiple GPUs?

After you have accelerated your application using a single GPU, it's natural to consider extending your app to take advantage of multiple GPUs in a single node or in multiple nodes of a GPU accelerated cluster.

Multiple GPUs can:

* Compute Faster - More GPUs equals faster time to a solution
* Compute Larger - More GPUs means more memory for larger problems
* Compute Cheaper - More GPUs per node translates to less overhead in money, power and space

Using CUDA-aware MPI with OpenACC allows you to efficiently utilize all the GPUs in a single node and also to scale across the GPUs in multiple nodes of a GPU accelerated cluster. If you are accelerating an (already) existing MPI parallel CPU code with OpenACC, going multi-GPU with CUDA-aware MPI and OpenACC is straight-forward.

## CUDA-aware MPI

A CUDA-aware MPI implementation allows you to exchange data directly to and from the four gpu buffers involved, avoiding host buffer staging in the user code. For this lab it is sufficient to know that you can directly pass GPU pointers to the MPI routines of a CUDA-aware MPI implementation. If you want to learn more about CUDA-aware MPI I recommend you to read my post on the Parallel Forall blog: 
[An Introduction to CUDA-Aware MPI](http://devblogs.nvidia.com/parallelforall/introduction-cuda-aware-mpi/)

## Structure of this Lab

This lab is broken up into three tasks; instructions for each will be provided in-line below. The "solution" to each task is the starting point of the next task, so you can skip around if you'd like. In addition, reference solutions are provided for each task. You can find them by looking for files matching \*.solution\*. You can build and run the solution with the make target <code>task?.solution</code>, e.g. for task 1: <code>make -C FORTRAN task1.solution</code>.

Instructions for downloading this IPython Notebook, as well as a .zip file of the source you worked on, are provided at the bottom of the lab in the <a href="#post-lab">Post Lab</a> section.

## Scalability Metrics For Success

The success of the multi GPU parallelization is measured with the following metrics. The provided tasks automatically print these metrics out at the end of each lab section/execution.

* Serial Time: $\Large{T_{s}}$  - how long it takes to run the problem with a single thread
* Parallel Time: $\Large{T_{p}}$  - how long it takes to run the problem in parallel
* Number of Processors:  $P$  - the number of processors operating in parallel
* Speedup: $\Large{S = \frac{T_{s}}{T_{p}}}$ - How much faster the parallel version is versus the serial version.
 * The ideal speed up is $P$.
* Efficiency: $\Large{E = \frac{S}{P}}$ - How efficiently the processors are being used.
 * The ideal efficiency is $1$.

## 2D Laplace Solver

The code used in this lab is a Jacobi solver for the 2D Laplace equation on a rectangle:


<div align="center">

$\LARGE{\Delta A(x,y) = 0 \; \forall (x,y) \in \Omega\backslash\delta\Omega}$

</div>

It uses Dirichlet boundary conditions (constant values on boundaries) on the left and right boundary and periodic boundary conditions on the top and bottom boundary. With the values chosen for left and right boundaries by the provided source code the solution looks like this

<div align="center">

<img src="files/jacobi_solution.PNG" width="50%" />

</div>

Given a 2D grid of vertexes, the solver attempts to set every vertex equal to the average of neighboring vertices.  It will iterate until the system converges to a stable value. So in each iteration of the Jacobi solver for all interior vertices 

<div align="center"><img src="files/laplace2d.PNG" width="30%" />$\LARGE{A_{k+1}(i,j)} = 0 - \frac{A_{k}(i-1,j) + A_{k}(i + 1,j) + A_{k}(i,j-1) + A_{k}(i,j+1)}{4}$</div>

is applied and then the periodic boundary conditions are handled by copying the values of the first interior row to the bottom boundary and the last interior row to the top boundary:

<div align="center">

<img src="files/jacobi_iteration_B.PNG" width="30%" />

</div>

# Lab Tasks

** This is a long lab, so please pay attention to your time.**  You have 120 minutes of access time from when the lab connection information was presented to you.  You may want to pre-download the IPython Notebook and source in the <a href="#post-lab">Post Lab</a> before continuing. When the tasks require to call a MPI routine the signature of the routine is given for you convenience but you need to pass in the right arguments to the routine (except <code>MPI_COMM_WORLD</code> and <code>MPI_STATUS_IGNORE</code>).

## Task #1

The purpose of this task is to show how to handle GPU affinity, add MPI boiler plate code in <code>FORTRAN/task1/laplace2d.F03</code> and make the necessary changes to <code>FORTRAN/task1/Makefile</code>. Look out for <code>TODO</code> in these two files. These will guide you through the following steps:

* <code>Makefile</code>: Start with MPI launcher (<code>mpirun –np …</code>)
* <code>laplace2d.F03</code>: Use MPI (<code>use mpi</code>)
* <code>laplace2d.F03</code>: Initialize MPI (<code>MPI_Init, MPI_Comm_rank, MPI_Comm_size</code>)
* <code>laplace2d.F03</code>: Handle GPU Affinity
* <code>laplace2d.F03</code>: Insert barriers to ensure correct timing (<code>MPI_Barrier</code>)
* <code>laplace2d.F03</code>: Finalize MPI (<code>MPI_Finalize</code>)

To compile and run simply issue <code>make -C FORTRAN task1</code> as given in the cell below.

The following reference might be interesting for you:

* API documentation for MPI from the OpenMPI website [https://www.open-mpi.org/doc/v1.8](https://www.open-mpi.org/doc/v1.8).
* [OpenACC 2.0 Quick Reference Guide](http://104.239.134.127/sites/default/files/213462%2010_OpenACC_API_QRG_HiRes.pdf)

[Click here to see hints](#Task-#1---Hints)

<iframe id="FORTRAN/task1" src="FORTRAN/task1" width="100%" height="600px">
  <p>Your browser does not support iframes.</p>
</iframe>

In [ ]:
!make -C FORTRAN task1

At the end of the output you will see a output similar to this:
```
Num GPUs: 4
4096x4096: 1 GPU:   5.2734 s, 4 GPUs:   5.1988 s, speedup:     1.01, efficiency:    25.36%
```
We are not getting any speed up although we are using four GPUs. The reason is that we simply quadrupled our work, that is, each GPU is solving the full problem. The alternative -- distributing the work across multiple GPUs by applying a domain decomposition -- is covered in task 2.

## Task #2

The purpose of this task is to apply a domain decomposition using horizontal stripes. To let you focus on the logical domain decomposition and GPU to GPU communication the data here is still fully replicated on each GPU. (This is something one would normally not do, as it has a significant storage overhead, but it avoids some extensive boilerplate code that should not be part of this lab.)

### Domain Decomposition

Here are three options for breaking up our 2D grid of vertexes, or domain, to parallelize the work across the multiple GPUs.  The halo region shown in light green in the images is the data that needs to be shared among the GPUs working on the problem.

<div style="display: inline;"><br><img src="files/decomp_tiles.PNG" align="left" width="15%" />
<ul style="position: relative; left: 15px; vertical-align: middle;">Minimizes surface area/volume ratio:
<li style="position: relative; left: 30px;">Communicate less data</li>
<li style="position: relative; left: 30px;">Optimal for bandwidth bound communication</li></ul></div>

<div style="clear: left;"><br><img src="files/decomp_vertical_stripes.PNG" align="left" width="15%" />
<ul style="position: relative; left: 15px;">Minimizes number of neighbors:
<li style="position: relative; left: 30px;">Communicate to fewer neighbors</li>
<li style="position: relative; left: 30px;">Optimal for latency bound communication</li>
Contiguous if data is [column-major](https://en.wikipedia.org/wiki/Column-major_order)</ul></div>

<div style="clear: left;"><br><img src="files/decomp_horizontal_stripes.PNG" align="left" width="15%" />
<br><ul style="position: relative; left: 15px;">Minimizes number of neighbors:
<li style="position: relative; left: 30px;">Communicate to fewer neighbors</li>
<li style="position: relative; left: 30px;">Optimal for latency bound communication</li>
Contiguous if data is [row-major](https://en.wikipedia.org/wiki/Row-major_order)</ul></div>

<div style="clear: left;"><br><br></div>

Due to the periodic boundary conditions at the top and the bottom of the domain we'll logically using the Horizontal Stripes decomposition. To avoid non contiguous communication for the column-major data layout of FORTRAN we will use 2D data arrays transposed. To do that we divide the size of the second dimension of our data array by the number of MPI ranks participating in a run and assign a chunk of that size to each rank. E.g. with a size of 500 for the second dimension and 2 MPI ranks: rank 0 would process from 2 to 250 and rank 1 from 251 to 499. In case the size of the second dimension is not divisible by the number of MPI ranks you need to take care of rounding errors.

### Halo updates

If the calculation is decomposed and distributed across the GPUs/MPI ranks each GPU/MPI rank works on its private copy of the data. To propagate information across the whole computation domain, we need to update the borders of each domain (so called halos) with the current values of the neighbouring GPUs/MPI ranks in each iteration. For example, rank <code>i</code> needs to sent the first row it has modified to the last row (the bottom domain boundary (halo)) of rank <code>i-1</code> (top neighbour), and the last row it has modified to the first row (the top domain boundary (halo)) of rank rank <code>i+1</code> (bottom neighbour):

<div align="center">

<img src="files/jacobi_halo_update_fortran.png" width="80%" />

</div>

If we treat rank <code>0</code> as the bottom neighbour of rank <code>n-1</code> and rank <code>n-1</code> as the top neighbour of rank <code>0</code> this step will also handle the periodic boundary conditions and therefore the loop handling for those can be removed.

### <code>!$acc host_data use_device( A )</code>
Since the iterations of the Jacobi solver are carried out on the GPUs the necessary halo updates also need to be done from and to the buffers in GPU memory. This is ensured with the directive <code>host_data use_device</code>, which was already inserted into the source code for you. <code>host_data use_device(A)</code> tells the OpenACC compiler to use the device representation of <code>A</code> in the following code block. Because we are using a CUDA-aware MPI the MPI implementation can handle these and do the halo updates directly to and from GPU memory. Without <code>host_data use_device(A)</code> the host representations of <code>A</code> would be passed to MPI and thus stale data would be exchanged between buffers in host memory which are not being used during the iterations of the Jacobi solver. If you want, you can try to remove the <code> host_data use_device(A)</code> directive and see what happens.

Like in Task #1 you should look out for <code>TODO</code> in <code>FORTRAN/task2/laplace2d.F03</code>. These will guide you through the following steps:

* Decompose the calculation across the GPUs/MPI ranks by adjusting the first and last row to be processed by each rank.
* Use <code>MPI_Allreduce</code> to calculate the global error across all GPUs/MPI ranks.
* Handle the periodic boundary conditions and the halo exchange with MPI as described above.

<iframe id="FORTRAN/task2" src="FORTRAN/task2" width="100%" height="600px">
  <p>Your browser does not support iframes.</p>
</iframe>

In [ ]:
!make -C FORTRAN task2

At the end of the output you will see a output similar to this:
```
Num GPUs: 4
4096x4096: 1 GPU:   5.2692 s, 4 GPUs:   1.6237 s, speedup:     3.25, efficiency:    81.13%
```
Now we are getting a speed up when using four GPUs. The efficiency is already quite decent but we can do better by hiding communication times. This will be covered in the next task.

## Optional: Using the NVIDIA Visual Profiler (NVVP)

As described in [CUDA Pro Tip: Profiling MPI Applications](http://devblogs.nvidia.com/parallelforall/cuda-pro-tip-profiling-mpi-applications/) <code>nvprof</code> can be used to generate profiles of MPI+OpenACC applications. Execute the <code>make task2.profile</code> in the cell below to generate a profile for each MPI rank executing your solution of task 2.

To view the generated profiles we'll be using the NVIDIA Visual Profiler (NVVP) tool which comes standard with the CUDA Toolkit software.  To launch the tool please <a href="/vnc" onclick="window.open(this.href, 'ProfilerVNC',
'left=20,top=20,width=1290,height=730,toolbar=1,resizable=0'); return false;">click here</a> which will open a new browser window.  **Note that it may take a few seconds for NVVP to start.**

After NVVP has started, import the generated profiles by clicking on "File" and then "Import...". In the dialog select "nvprof" and "Multiple Processes". Browse to <code>ubuntu/notebook/FORTRAN/task2</code> and select <code>laplace2d.[0-3].nvvp</code>.

If you've never used NVVP before or if you want to read more about you can [click here](https://developer.nvidia.com/nvidia-visual-profiler) for more information.

In [ ]:
!make -C FORTRAN task2.profile

## Task #3

By applying a domain decomposition and distributing the work across multiple GPUs in task #2 we could get a speed-up but do not attain optimal efficiency. This is because of the time that is needed (spent) to carry out the halo updates using MPI. This wasted time is called "parallel overhead" because it is a step not necessary for execution with a single GPU. We can lower the parallel overhead by doing computations in parallel with the MPI communication and therefore hide the communication time. In the case of our Jacobi solver, this is best done by splitting each domain into a boundary part (which updates all values that we need to communicate) and an inner part. By doing this split, we can start the MPI communication after the boundary part has finished, and let it run in parallel with the inner part:

<div align="center">

<img src="files/overlapping_mpi_compute.png" width="80%" />

</div>

In OpenACC this can be done by using the async clause on a kernels region as outlined below.

```FORTRAN
!$acc kernels
DO
     !Process boundary
END DO
!$acc end kernels

!$acc kernels async
DO
     !Process boundary
END DO
!$acc end kernels

!$acc host_data use_device( a )
  !Exchange halo with top and bottom neighbor
  CALL MPI_Sendrecv( a…)
  !…
!$acc end host_data
!wait for iteration to finish
!$acc wait
```

In this task you should apply this approach to the copy loop of the Jacobi solver. As in in the earlier tasks in this lab, you should look for <code>TODO</code> in <code>FORTRAN/task3/laplace2d.F03</code>. These will guide you through the following steps:

* Split the copy loop into its constituent halo and bulk parts.
* Start the computation of the bulk part asynchronously.
* Wait for the bulk part to complete at the end of the iteration.

<iframe id="FORTRAN/task3" src="FORTRAN/task3" width="100%" height="600px">
  <p>Your browser does not support iframes.</p>
</iframe>

In [ ]:
!make -C FORTRAN task3

## Summary

In this lab you have learned that using CUDA-aware MPI in combination with OpenACC is a effective way to exploit the power of multiple GPUs in a node or a cluster.

<a id="post-lab"></a>
## Post-Lab

Finally, don't forget to save your work from this lab before time runs out and the instance shuts down!!

1. Save this IPython Notebook by going to `File -> Download as -> IPython (.ipynb)` at the top of this window
2. You can execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
rm -f multi_gpu_mpi_openacc_files.zip
zip -r multi_gpu_mpi_openacc_files.zip C FORTRAN Makefile

**After** executing the above cell, you should be able to download the zip file [here](files/multi_gpu_mpi_openacc_files.zip)

<a id="References/Further Reading"></a>
## References/Further Reading

* Learn more at the [CUDA Developer Zone](https://developer.nvidia.com/category/zone/cuda-zone).
* If you have an NVIDIA GPU in your system, you can download and install the [CUDA tookit](https://developer.nvidia.com/cuda-toolkit).
* Take the fantastic online and **free** Udacity [Intro to Parallel Programming](https://www.udacity.com/course/cs344) course which uses CUDA C.
* Search or ask questions on [Stackoverflow](http://stackoverflow.com/questions/tagged/cuda) using the cuda tag
* Read the GPU Computing developer blog [Parallel Forall](http://devblogs.nvidia.com/parallelforall/)

<a id="FAQ"></a>
---
# Lab FAQ

Q: I'm encountering issues executing the cells, or other technical problems?<br>
A: Please see [this](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) infrastructure FAQ.

## Hints

### Task #1 - Hints
#### Hint #1
To handle GPU affinity we assign each MPI rank to one GPU so you need to map the rank to the id of the device to use.
 
#### Hint #2
You can assume that MPI ranks are started consecutively on each node. I.e., with 8 processes on 2 nodes rank 0,1,2, and 3 are started on node 0 and rank 4,5,6,7 are started on node 1.

#### Hint #3
You can use code like the following to handle the GPU affinity.
```
#if _OPENACC
    ngpus=acc_get_num_devices( acc_device_nvidia )
    devicenum = MOD( mpi_rank, ngpus )
    call acc_set_device_num( devicenum, acc_device_nvidia )
    call acc_init( acc_device_nvidia )
#endif
```

[Return to Task #1](#Task-#1)

<style>
p.hint_trigger{
  margin-bottom:7px;
  margin-top:-5px;
  background:#64E84D;
}
.toggle_container{
  margin-bottom:0px;
}
.toggle_container p{
  margin:2px;
}
.toggle_container{
  background:#f0f0f0;
  clear: both;
  font-size:100%;
}
</style>
<script>
$("p.hint_trigger").click(function(){
   $(this).toggleClass("active").next().slideToggle("normal");
});
   
$(".toggle_container").hide();
</script>